<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section_3/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストデータの前処理
torchtextを使って、テキストデータをニューラルネットワークへの入力に適した形に整えます。


## 各設定
### APIのkey
Twitter Developer PlatformからAPI Key & SecretおよびAccess Token & Secretを取得し、以下のセルの変数に設定します。
```
api_key ="API key"
api_key_secret ="API key secret"
access_token="Access token"
access_token_secret ="Access token secret"
``` 

In [ ]:
api_key =
api_key_secret =
access_token =
access_token_secret =

## APIの設定
Tweepyを利用し、OAuth認証にアクセストークンを設定します。  
その上で、Twitterとやり取りするためのAPIを設定します。

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### ライブラリののインストール
分かち書きのためにjanomeを、テキストデータの前処理のためにtorchtextをインストールします。

In [ ]:
!pip install janome==0.4.1
!pip install -U torchtext==0.7.0

## Tweetの取得と保存
タイムラインのツイートを取得し、csvファイルとして保存します。  

In [ ]:
import csv

user_id = "live_ai_exp"
tweets = api.user_timeline(user_id, count=10)
texts = [[tweet.text] for tweet in tweets]  # 内包表記
with open("texts.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(texts)

保存されたcsvファイルを読み込んで、内容を確認しましょう。

In [ ]:
with open("texts.csv") as f:
    print(f.read())

## データセットの生成
Fieldを指定してcsvファイルを読み込み、データセットを生成します。

In [ ]:
import torchtext
from janome.tokenizer import Tokenizer

j_tk = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_tk.tokenize(text, wakati=True)]  # 
 
# データセットの列を定義
text_field = torchtext.data.Field(
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    batch_first=True,  # アルファベットを小文字に変換
    lower=True  # バッチの次元を先頭に
    )
 
# csvファイルからデータセットを作成
train_dataset = torchtext.data.TabularDataset(
    path="texts.csv",
    format="csv",
    fields=[("tw_text", text_field)]  # 列の設定
    )

# 列の内容を表示
for example in train_dataset.examples:
    print(example.tw_text)

## 単語の埋め込みベクトル化
FastTextを使って、単語を埋め込みベクトルに変換します。

In [ ]:
from torchtext.vocab import FastText

text_field.build_vocab(train_dataset, vectors=FastText(language="ja"))
print(text_field.vocab.freqs)  # 各単語の出現頻度
print()
print(text_field.vocab.stoi)  # 各単語に振られたインデックス
print()
print(text_field.vocab.vectors.size())  # ベクトルのサイズ
print()
print(text_field.vocab.vectors[3])  # ベクトルの表示

## Iteratorの設定
バッチごとに学習を行うために、Iteratorを設定します。  
これを使えば、データセットから指定したバッチサイズのミニバッチを取り出すことができます。

In [ ]:
# Iteratorの設定
train_iterator = torchtext.data.Iterator(
    train_dataset,
    batch_size=8, 
    train=True  # シャッフルして取り出す
)

ミニバッチを取り出して、内容を表示します。  
ミニバッチには、単語をインデックスに置き換えた文章が格納されます。



In [ ]:
batch = next(iter(train_iterator))  # ミニバッチを取り出す
print(batch.tw_text.size())  # ミニバッチのサイズ
print(batch.tw_text[0])  # 最初の要素

## 単語埋め込み層の実装
単語を埋め込みベクトルに変換する層を実装します。

In [ ]:
import torch

class WordEmbedding(torch.nn.Module):
    def __init__(self, vectors):
        super().__init__()

        # 埋め込み層
        self.embedding = torch.nn.Embedding.from_pretrained(
            embeddings=vectors, freeze=True
        )

    def forward(self, x):
        x = self.embedding(x)  # 単語をベクトルに変換
        return x
    
word_embedding = WordEmbedding(text_field.vocab.vectors)

x = batch.tw_text
print(x.size())
print(x[0])

y = word_embedding(x)
print(y.size())
print(y[0])